In [38]:
import time
import math
import tensorflow as tf
import keras 
import numpy as np
import pandas as pd
from keras.models import load_model,Model
from keras.engine.topology import Layer

In [2]:
# 定义融合层，将深度学习算法与历史均值算法融合
class Merge_Layer(Layer):
    def __init__(self, **kwargs):
        super(Merge_Layer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.para1 = self.add_weight(shape=(input_shape[0][1], input_shape[0][2]),
                                     initializer='uniform', trainable=True,
                                     name='para1')
        self.para2 = self.add_weight(shape=(input_shape[1][1], input_shape[1][2]),
                                     initializer='uniform', trainable=True,
                                     name='para2')
        super(Merge_Layer, self).build(input_shape)

    def call(self, inputs):
        mat1 = inputs[0]
        mat2 = inputs[1]
        output = mat1 * self.para1 + mat2 * self.para2
        # output = mat1 * 0.1 + mat2 * 0.9
        return output

    def compute_output_shape(self, input_shape):
        return input_shape[0]

In [3]:
#定义精度评价指标。为防止0值附近相对误差过大而导致的异常，定义mask层。
def mape_loss_func(preds, labels):
    mask = labels > 5
    return np.mean(np.fabs(labels[mask]-preds[mask])/labels[mask])

def smape_loss_func(preds, labels):
    mask= labels > 5
    return np.mean(2*np.fabs(labels[mask]-preds[mask])/(np.fabs(labels[mask])+np.fabs(preds[mask])))

def mae_loss_func(preds, labels):
    mask= labels > 5
    return np.fabs((labels[mask]-preds[mask])).mean()

def eliminate_nan(b):
    a = np.array(b)
    c = a[~np.isnan(a)]
    return c

In [39]:
near_road = np.array(pd.read_csv('2small_network_nearest_road_id.csv',header = None))
flow = np.array(pd.read_csv(r"2small_network_speed.csv", header= None)) #注意header=0 or None

# 利用滑动窗口的方式，重构数据为(n，最近路段数，输入时间窗，总路段数)的形式

time3 = time.time()

k = 5 # 参数k为需考虑的最近路段数
t_p = 29 # 参数t_p为总时间序列长度（天）
t_input = 12 #参数t_input为输入时间窗(5min颗粒度)
t_pre = 3 #参数t_pre为预测时间窗(5min颗粒度)
num_links = 30 #参数num_links为总路段数


image = []
for i in range(np.shape(near_road)[0]):
    road_id = []
    for j in range(k):
        road_id.append(near_road[i][j])
    image.append(flow[road_id, :])
image1 = np.reshape(image, [-1, k, len(flow[0,:])])
image2 = np.transpose(image1,(1,2,0))
image3 = []
label = []
day = []

for i in range(1,t_p):
    for j in range(180-t_input-t_pre):
        image3.append(image2[:, i*180+j:i*180+j+t_input, :][:])
        label.append(flow[:, i*180+j+t_input:i*180+j+t_input+t_pre][:])
        day.append(flow[:, (i-1)*180+j+t_input:(i-1)*180+j+t_input+t_pre][:])

image3 = np.asarray(image3)
label = np.asarray(label)
day =  np.asarray(day)

print(np.shape(image3))
print(np.shape(label))
print(np.shape(day))

#划分前80%数据为训练集，最后20%数据为测试集
image_train_source = image3[:np.shape(image3)[0]*1//10]
image_test_source = image3[np.shape(image3)[0]*1//10:]
label_train_source = label[:np.shape(label)[0]*1//10]
label_test_source = label[np.shape(label)[0]*1//10:]

day_train_source = day[:np.shape(day)[0]*1//10]
day_test_source = day[np.shape(day)[0]*1//10:]


time4 = time.time()
print('input done %g' % (time4-time3))

(4620, 5, 12, 30)
(4620, 30, 3)
(4620, 30, 3)
input done 0.0299215


In [40]:
near_road = np.array(pd.read_csv('small_network_nearest_road_id.csv',header = None))
flow = np.array(pd.read_csv(r"small_network_speed.csv", header= None)) #注意header=0 or None
# 利用滑动窗口的方式，重构数据为(n，最近路段数，输入时间窗，总路段数)的形式

time3 = time.time()

k = 5 # 参数k为需考虑的最近路段数
t_p = 29 # 参数t_p为总时间序列长度（天）
t_input = 12 #参数t_input为输入时间窗(5min颗粒度)
t_pre = 3 #参数t_pre为预测时间窗(5min颗粒度)
num_links = 30 #参数num_links为总路段数


image = []
for i in range(np.shape(near_road)[0]):
    road_id = []
    for j in range(k):
        road_id.append(near_road[i][j])
    image.append(flow[road_id, :])
image1 = np.reshape(image, [-1, k, len(flow[0,:])])
image2 = np.transpose(image1,(1,2,0))
image3 = []
label = []
day = []

for i in range(1,t_p):
    for j in range(180-t_input-t_pre):
        image3.append(image2[:, i*180+j:i*180+j+t_input, :][:])
        label.append(flow[:, i*180+j+t_input:i*180+j+t_input+t_pre][:])
        day.append(flow[:, (i-1)*180+j+t_input:(i-1)*180+j+t_input+t_pre][:])

image3 = np.asarray(image3)
label = np.asarray(label)
day =  np.asarray(day)

print(np.shape(image3))
print(np.shape(label))
print(np.shape(day))

#划分前80%数据为训练集，最后20%数据为测试集
image_train_target = image3[:np.shape(image3)[0]*1//10]
image_test_target = image3[np.shape(image3)[0]*1//10:]
label_train_target = label[:np.shape(label)[0]*1//10]
label_test_target = label[np.shape(label)[0]*1//10:]

day_train_target = day[:np.shape(day)[0]*1//10]
day_test_target = day[np.shape(day)[0]*1//10:]


time4 = time.time()
print('input done %g' % (time4-time3))

(4620, 5, 12, 30)
(4620, 30, 3)
(4620, 30, 3)
input done 0.0289218


In [7]:
#模型构建
input_data = keras.Input(shape=(k,t_input,num_links), name='input_data')
input_HA = keras.Input(shape=(num_links, t_pre), name='input_HA')

x = keras.layers.BatchNormalization(input_shape =(k,t_input,num_links))(input_data)

x = keras.layers.Conv2D(
                           filters = 30,
                           kernel_size = 3,
                           strides = 1,
                           padding="SAME",
                           activation='relu')(x)

x = keras.layers.AveragePooling2D(pool_size = (2,2),
                                strides = 1,
                                padding = "SAME",
                                )(x)

x = keras.layers.BatchNormalization()(x)

x = keras.layers.Conv2D(
                       filters = 30,
                       kernel_size = 3,
                       strides = 1,
                       padding="SAME",
                       activation='relu')(x)

x = keras.layers.AveragePooling2D(pool_size = (2,2),
                                strides = 1,
                                padding = "SAME",
                                )(x)
x = keras.layers.Flatten()(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dropout(0.5)(x)
x = keras.layers.Dense(num_links*t_pre, activation='relu')(x)

output = keras.layers.Reshape((num_links,t_pre))(x)

output_final = Merge_Layer()([output, input_HA])

# construct model
finish_model = keras.models.Model([input_data,input_HA], [output_final])

finish_model.summary()


n\Python37\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_data (InputLayer)         (None, 5, 12, 30)    0                                            
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 5, 12, 30)    120         input_data[0][0]                 
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 5, 12, 30)    8130        batch_normalization_1[0][0]      
_____________________

In [8]:
#参数加载
finish_model.load_weights('source.h5')


In [18]:
#模型预测
model_pre = finish_model.predict([image_test,day_test])


In [19]:
#预测结果存储
# model_pre = np.reshape(model_pre,[103, 6])
# model_pre1 = pd.DataFrame(model_pre)
# model_pre1.to_csv('预测值.csv', index = False)

In [20]:
#transfer without FT 预测精度计算

mape_mean = mape_loss_func(model_pre, label_test)
smape_mean = smape_loss_func(model_pre, label_test)
mae_mean = mae_loss_func(model_pre, label_test)

print('mape = ' + str(mape_mean) + '\n' + 'smape = ' + str(smape_mean) + '\n' + 'mae = ' + str(mae_mean))

mape = 0.7964474115108172
smape = 1.343935013674002
mae = 21.05578354714718


In [29]:

middle = Model(inputs=[input_data, input_HA],outputs=finish_model.get_layer('dense_1').output)

In [41]:
middle_result_source = middle.predict([image_train_source, day_train_source])
middle_result_target = middle.predict([image_train_target, day_train_target])

In [46]:
from sklearn import metrics
def mmd (x, y):
    return metrics.mean_squared_error(x,y)

In [47]:
mmd (middle_result_source, middle_result_target)

9859.165

In [50]:
from keras import backend as K

In [51]:
loss1 = K.mean(K.square(output_final - label_train_target), axis=-1) 
loss2 = 0.001 * mmd (middle_result_source, middle_result_target)
overall_loss = loss1 + loss2

In [53]:
def new_loss(output_final, label_train_target):
    middle = Model(inputs=[input_data, input_HA],outputs=finish_model.get_layer('dense_1').output)
    middle_result_source = middle.predict([image_train_source, day_train_source])
    middle_result_target = middle.predict([image_train_target, day_train_target])

    loss1 = K.mean(K.square(output_final - label_train_target), axis=-1) 
    loss2 = 0.001 * mmd (middle_result_source, middle_result_target)
    overall_loss = loss1 + loss2
    return overall_loss


In [54]:
finish_model.compile(optimizer='adam',loss=new_loss)

In [64]:
finish_model.fit([image_train_target, day_train_target], label_train_target, epochs=100, batch_size=462,
validation_data=([image_test_target,day_test_target], label_test_target))

al_loss: 48.3029
Epoch 54/100
462/462 [==============================] - 0s 138us/step - loss: 30.2374 - val_loss: 48.3742
Epoch 55/100
462/462 [==============================] - 0s 142us/step - loss: 30.2183 - val_loss: 48.4435
Epoch 56/100
462/462 [==============================] - 0s 143us/step - loss: 30.2798 - val_loss: 48.5070
Epoch 57/100
462/462 [==============================] - 0s 143us/step - loss: 29.9690 - val_loss: 48.5410
Epoch 58/100
462/462 [==============================] - 0s 142us/step - loss: 29.9869 - val_loss: 48.5616
Epoch 59/100
462/462 [==============================] - 0s 143us/step - loss: 30.0396 - val_loss: 48.5686
Epoch 60/100
462/462 [==============================] - 0s 142us/step - loss: 30.0838 - val_loss: 48.5433
Epoch 61/100
462/462 [==============================] - 0s 144us/step - loss: 29.9392 - val_loss: 48.5133
Epoch 62/100
462/462 [==============================] - 0s 144us/step - loss: 29.8501 - val_loss: 48.4758
Epoch 63/100
462/462 [=======

In [65]:
model_pre = finish_model.predict([image_test_target,day_test_target])

In [66]:
#transfer with DAN 预测精度计算

mape_mean = mape_loss_func(model_pre, label_test)
smape_mean = smape_loss_func(model_pre, label_test)
mae_mean = mae_loss_func(model_pre, label_test)

print('mape = ' + str(mape_mean) + '\n' + 'smape = ' + str(smape_mean) + '\n' + 'mae = ' + str(mae_mean))

mape = 0.21396551677012662
smape = 0.18441804844797388
mae = 4.60459914071357


In [67]:
mape_list = []
for i in range(num_links):
    a1 = mape_loss_func(model_pre[:,i,:], label_test[:,i,:])
    mape_list.append(a1)

mape_pd = pd.Series(mape_list)
mape_pd.sort_values()

25    0.079514
0     0.092317
1     0.096132
7     0.102632
8     0.118572
17    0.119644
6     0.131133
20    0.131280
15    0.134143
3     0.136437
14    0.139297
12    0.156464
5     0.159403
2     0.174019
21    0.183724
10    0.184623
19    0.193801
28    0.210484
11    0.224735
9     0.225280
18    0.261624
13    0.303682
27    0.319340
24    0.334712
29    0.335376
23    0.350989
26    0.359598
4     0.365386
16    0.379909
22    0.415017
dtype: float64